### ⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
# Laboratory 1 Solution
### Cooperating with: Riccardo Vaccari (s348856)
### ⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯

## Import

In [52]:
import numpy as np
from tqdm.auto import tqdm

#also use complete random guess 20% or heuristic 80%

## Setup

In [1]:
NUM_SAMPLES = 10  # number of candidates solution
MAX_STEPS = 10000

In [14]:
# Problem 1 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))


In [33]:
# Problem 2 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS,  size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

In [49]:
# Problem 3 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS,  size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

In [ ]:
# fitness function
def fitness(sol):
    return VALUES[np.any(sol, axis=0)].sum()

# validity check
def is_valid(sol):
    # each item in at most one knapsack
    if np.any(sol.sum(axis=0) > 1):
        return False
    
    # each knapsack respects its constraints
    return np.all([WEIGHTS[sol[k]].sum(axis=0) <= CONSTRAINTS[k] for k in range(NUM_KNAPSACKS)])

# tweak: flip one random item in one random knapsack
def tweak(sol):
    new_solution = sol.copy()
    k = rng.integers(0, NUM_KNAPSACKS)
    i = rng.integers(0, NUM_ITEMS)
    
    # flip presence
    new_solution[k, i] = not new_solution[k, i]

    return new_solution

"""
# COMMENTED BECAUSE RETURN WORST RESULTS THAN BASIC TWEAK ABOVE

# Generate a new candidate solution by randomly adding or removing items across knapsacks. 
# The modification repeats while rng.random() < t, allowing multiple changes per tweak.
# At each steps, remove all items from knapsack and k can be:
# • < NUM_KNAPSACKS: the item is added to knapsack k.
# • == NUM_KNAPSACKS: the item remains unassigned.

def tweak(sol, t = 0.6):
    new_solution = sol.copy()

    while rng.random() < t:
        k = rng.integers(0, NUM_KNAPSACKS+1)
        i = rng.integers(0, NUM_ITEMS)

        new_solution[:, i] = False

        if k < NUM_KNAPSACKS:
            new_solution[k, i] = True

    return new_solution

"""

## Base Hill Climbing

In [ ]:
def baseHillClimbing(solution):
    for _ in tqdm(range(MAX_STEPS)):
        new_solution = tweak(solution)

        if not is_valid(new_solution):
            continue
        
        # accept if equal or better to avoid remaining stuck
        if fitness(new_solution) >= fitness(solution):
            solution = new_solution
    
    return solution

## Hill Climbing (with Tabu Search)

In [ ]:
def tabuSearchHillClimbing(solution):
    tabu_list = set()

    for _ in tqdm(range(MAX_STEPS)):
        possible_solutions = [tweak(solution) for _ in range(NUM_SAMPLES)]
        candidates = [
            sol for sol in possible_solutions 
            if is_valid(sol) and sol.flatten().tobytes() not in tabu_list
        ]

        if not candidates:
            continue

        new_solution = max(candidates, key=fitness)

        # accept if equal or better to avoid remaining stuck
        if fitness(new_solution) >= fitness(solution):
            tabu_list.add(solution.flatten().tobytes())
            solution = new_solution
    return solution    


## Results

In [51]:
# start with empty solution (no items assigned)
first_guess = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)

# Results 
solution = tabuSearchHillClimbing(first_guess)
total_value = fitness(solution)
values_per_knapsack = [VALUES[solution[k]].sum() for k in range(NUM_KNAPSACKS)]

print("Total value:", total_value)
for k, v in enumerate(values_per_knapsack):
    print(f"Knapsack {k+1} value: {v}")

for k in range(NUM_KNAPSACKS):
    total_weights = WEIGHTS[solution[k]].sum(axis=0)
    print(f"Knapsack {k+1} weights: {total_weights}, constraints: {CONSTRAINTS[k]}")

100%|██████████| 10000/10000 [00:55<00:00, 181.52it/s]

Total value: 1644698
Knapsack 1 value: 19243
Knapsack 2 value: 18730
Knapsack 3 value: 14169
Knapsack 4 value: 19652
Knapsack 5 value: 12820
Knapsack 6 value: 17844
Knapsack 7 value: 17840
Knapsack 8 value: 16051
Knapsack 9 value: 15042
Knapsack 10 value: 14632
Knapsack 11 value: 14648
Knapsack 12 value: 16732
Knapsack 13 value: 15388
Knapsack 14 value: 15249
Knapsack 15 value: 18956
Knapsack 16 value: 15348
Knapsack 17 value: 13752
Knapsack 18 value: 17228
Knapsack 19 value: 17830
Knapsack 20 value: 15286
Knapsack 21 value: 16508
Knapsack 22 value: 19985
Knapsack 23 value: 17519
Knapsack 24 value: 14883
Knapsack 25 value: 14408
Knapsack 26 value: 16703
Knapsack 27 value: 18212
Knapsack 28 value: 18548
Knapsack 29 value: 14787
Knapsack 30 value: 18960
Knapsack 31 value: 16821
Knapsack 32 value: 13098
Knapsack 33 value: 16244
Knapsack 34 value: 17725
Knapsack 35 value: 20962
Knapsack 36 value: 17612
Knapsack 37 value: 12684
Knapsack 38 value: 18227
Knapsack 39 value: 15808
Knapsack 40 v